# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 22 2022 10:26AM,240171,10,MSP211501,"Methapharm, Inc.",Released
1,Jun 22 2022 10:26AM,240171,10,MSP211502,"Methapharm, Inc.",Released
2,Jun 22 2022 10:26AM,240171,10,MSP211503,"Methapharm, Inc.",Released
3,Jun 22 2022 10:26AM,240171,10,MSP211504,"Methapharm, Inc.",Released
4,Jun 22 2022 10:26AM,240171,10,MSP211507,"Methapharm, Inc.",Released
5,Jun 22 2022 10:26AM,240171,10,MSP211508,"Methapharm, Inc.",Released
6,Jun 22 2022 10:26AM,240171,10,MSP211512,"Methapharm, Inc.",Released
7,Jun 22 2022 10:26AM,240171,10,MSP211513,"Methapharm, Inc.",Released
8,Jun 22 2022 10:24AM,240170,19,DEX0007039,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,Jun 22 2022 10:24AM,240170,19,DEX0007040,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,240165,Released,1
46,240166,Released,1
47,240169,Released,2
48,240170,Released,2
49,240171,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240165,NaN,NaN,1.0
240166,NaN,NaN,1.0
240169,NaN,NaN,2.0
240170,NaN,NaN,2.0
240171,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240034,0.0,0.0,9.0
240079,1.0,10.0,1.0
240081,0.0,35.0,2.0
240085,0.0,8.0,4.0
240086,0.0,7.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240034,0,0,9
240079,1,10,1
240081,0,35,2
240085,0,8,4
240086,0,7,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240034,0,0,9
1,240079,1,10,1
2,240081,0,35,2
3,240085,0,8,4
4,240086,0,7,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240034,,,9
1,240079,1,10,1
2,240081,,35,2
3,240085,,8,4
4,240086,,7,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 22 2022 10:26AM,240171,10,"Methapharm, Inc."
8,Jun 22 2022 10:24AM,240170,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
10,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC"
12,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC"
13,Jun 22 2022 10:16AM,240165,10,Bio-PRF
14,Jun 22 2022 10:09AM,240157,10,Bio-PRF
41,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc."
68,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation
80,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.
82,Jun 22 2022 9:55AM,240161,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 22 2022 10:26AM,240171,10,"Methapharm, Inc.",,,8
1,Jun 22 2022 10:24AM,240170,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,2
2,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",,,2
3,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",,,1
4,Jun 22 2022 10:16AM,240165,10,Bio-PRF,,,1
5,Jun 22 2022 10:09AM,240157,10,Bio-PRF,,,27
6,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",,,27
7,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,,,12
8,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,,,2
9,Jun 22 2022 9:55AM,240161,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:26AM,240171,10,"Methapharm, Inc.",8,,
1,Jun 22 2022 10:24AM,240170,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
2,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",2,,
3,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",1,,
4,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1,,
5,Jun 22 2022 10:09AM,240157,10,Bio-PRF,27,,
6,Jun 22 2022 10:09AM,240164,15,"Person & Covey, Inc.",27,,
7,Jun 22 2022 10:04AM,240160,10,ISDIN Corporation,12,,
8,Jun 22 2022 10:00AM,240163,10,Eywa Pharma Inc.,2,,
9,Jun 22 2022 9:55AM,240161,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:26AM,240171,10,"Methapharm, Inc.",8,,
1,Jun 22 2022 10:24AM,240170,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
2,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",2,,
3,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",1,,
4,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:26AM,240171,10,"Methapharm, Inc.",8.0,NaN,NaN
1,Jun 22 2022 10:24AM,240170,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,NaN,NaN
2,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN,NaN
3,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",1.0,NaN,NaN
4,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 22 2022 10:26AM,240171,10,"Methapharm, Inc.",8.0,0.0,0.0
1,Jun 22 2022 10:24AM,240170,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,0.0,0.0
2,Jun 22 2022 10:23AM,240169,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0,0.0
3,Jun 22 2022 10:22AM,240166,15,"Brookfield Pharmaceuticals, LLC",1.0,0.0,0.0
4,Jun 22 2022 10:16AM,240165,10,Bio-PRF,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3121788,77.0,14.0,1.0
102,240089,1.0,0.0,0.0
12,480311,1.0,15.0,0.0
15,1200769,100.0,0.0,0.0
16,480275,1.0,6.0,0.0
19,2161202,15.0,22.0,3.0
20,1440644,22.0,67.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3121788,77.0,14.0,1.0
1,102,240089,1.0,0.0,0.0
2,12,480311,1.0,15.0,0.0
3,15,1200769,100.0,0.0,0.0
4,16,480275,1.0,6.0,0.0
5,19,2161202,15.0,22.0,3.0
6,20,1440644,22.0,67.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77.0,14.0,1.0
1,102,1.0,0.0,0.0
2,12,1.0,15.0,0.0
3,15,100.0,0.0,0.0
4,16,1.0,6.0,0.0
5,19,15.0,22.0,3.0
6,20,22.0,67.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,100.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20
Status,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0
Executing,14.0,0.0,15.0,0.0,6.0,22.0,67.0
Released,77.0,1.0,1.0,100.0,1.0,15.0,22.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,14.0,0.0,15.0,0.0,6.0,22.0,67.0
2,Released,77.0,1.0,1.0,100.0,1.0,15.0,22.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20
0,Completed,1.0,0.0,0.0,0.0,0.0,3.0,0.0
1,Executing,14.0,0.0,15.0,0.0,6.0,22.0,67.0
2,Released,77.0,1.0,1.0,100.0,1.0,15.0,22.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()